# Mass Transfer Lesson: Designing a Liquid-Liquid Extraction Cascade

**Objective:** This lesson will guide you through the design of a multi-stage, counter-current liquid-liquid extraction (LLE) process. We will use a stage-by-stage calculation method, analogous to McCabe-Thiele, to determine the number of theoretical stages needed.

**Learning Goals:**
1.  Understand the principles of LLE and the role of the partition coefficient.
2.  Derive and apply the operating line equation for a counter-current cascade.
3.  Construct a graphical representation of the LLE process (an x-y diagram).
4.  Implement a computational algorithm to step off the theoretical stages.
5.  Analyze how changing the solvent-to-feed ratio impacts the separation.

## The Process: Counter-Current Extraction

We have a **feed** stream containing a valuable **solute (A)** dissolved in a **carrier (C)**. We want to extract the solute using a **solvent (S)** that is immiscible with the carrier. The solute-depleted carrier stream leaving is called the **raffinate**, and the solute-rich solvent stream leaving is called the **extract**.

In a counter-current cascade, the feed enters at one end (Stage 1) and the solvent enters at the other (Stage N). This is the most efficient configuration.

![LLE Diagram](https://ars.els-cdn.com/content/image/3-s2.0-B9780080966595000037-f03-12-9780080966595.jpg)

**Our Goal:** To reduce the concentration of solute in the feed down to a target concentration in the final raffinate, determine how many theoretical stages are required.

## The Model: Equilibrium and Operating Lines

Let $x$ be the mass fraction of solute in the raffinate phase (carrier + solute) and $y$ be the mass fraction in the extract phase (solvent + solute). For this simplified model, we will assume the carrier and solvent are completely immiscible.

**1. Equilibrium Line:** The thermodynamic relationship between the phases at equilibrium. We will assume a constant **Partition Coefficient ($K_D$)**.
$$ y = K_D x $$

**2. Operating Line:** Derived from a mass balance. It relates the concentration of the raffinate stream entering a stage to the extract stream leaving that same stage.
$$ y_{n} = \frac{L_s}{V_s} (x_{n-1} - x_{N}) + y_{N+1} $$
Where $L_s$ is the carrier flow rate (kg/hr), $V_s$ is the solvent flow rate (kg/hr), $x_N$ is the final raffinate concentration, and $y_{N+1}$ is the fresh solvent concentration (usually 0).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Define System Parameters ---
F = 100.0         # Feed flow rate (kg/hr) - Carrier + Solute
S = 150.0         # Solvent flow rate (kg/hr)
xF = 0.20         # Mass fraction of solute in Feed
xN_target = 0.01  # Desired mass fraction of solute in final Raffinate
yS = 0.00         # Mass fraction of solute in fresh Solvent

# Equilibrium Data
KD = 5.0          # Partition Coefficient (y/x)

print("System parameters are defined.")

In [ ]:
# --- Stage Calculation Algorithm ---

# Calculate the flow rates of the pure carrier and pure solvent (assumed constant)
Ls = F * (1 - xF) # Carrier flow rate
Vs = S * (1 - yS) # Solvent flow rate

# Overall material balance on the solute to find the exit extract concentration, y1
# F*xF + S*yS = (Ls + Ls*xN)*xN + (Vs + Vs*y1)*y1 -- this is too complex
# Simpler balance: Solute_in = Solute_out
# F*xF + S*yS = (Ls*xN_target/(1-xN_target)) + (Vs*y1/(1-y1))
# Let's use mass flows: F_solute_in = R_solute_out + E_solute_out
# F*xF = Ls*xN_target + Vs*y1  (This is an approximation assuming x and y are small)
y1 = (F * xF - Ls * xN_target) / Vs

# Setup for stepping off stages graphically
x_steps = []
y_steps = []
num_stages = 0

# Start at stage 1 (feed end)
y_current = y1
x_current = y_current / KD # x1 is in equilibrium with y1
x_steps.extend([x_current])
y_steps.extend([y_current])

while x_current > xN_target:
    num_stages += 1
    
    # Point on the operating line
    # y_n+1 = (Ls/Vs)*(x_n - xN) + y_N+1
    # We know x_n, we need to find y_n+1
    y_next = (Ls/Vs)*(x_current - xN_target) + yS
    x_steps.extend([x_current, x_current])
    y_steps.extend([y_current, y_next])
    
    # Point on the equilibrium line
    y_current = y_next
    x_current = y_current / KD
    x_steps.extend([x_current])
    y_steps.extend([y_current])
    
    # Safety break
    if num_stages > 50:
        print("Exceeded 50 stages, breaking loop.")
        break

print(f"Calculation Complete! Required theoretical stages: {num_stages}")

In [ ]:
# --- Plotting the LLE Diagram ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(10, 10))

# Define range for lines
x_range = np.linspace(0, 0.25, 100)
y_range_eq = KD * x_range

# Operating Line: connects (xN, yS) to (x1, y1)
x1 = y1 / KD
op_line_x = [xN_target, x1]
op_line_y = [yS, y1]

plt.plot(x_range, y_range_eq, 'b-', label=f'Equilibrium Line (K_D={KD})')
plt.plot(op_line_x, op_line_y, 'g-', label='Operating Line')
plt.plot(x_steps, y_steps, 'r-o', markersize=4, label=f'{num_stages} Stages')

# Formatting
plt.title('LLE Counter-Current Cascade Design', fontsize=16, weight='bold')
plt.xlabel('Mass Fraction in Raffinate, x', fontsize=12)
plt.ylabel('Mass Fraction in Extract, y', fontsize=12)
plt.xlim(0, x1 * 1.1)
plt.ylim(0, y1 * 1.1)
plt.legend()
plt.grid(True)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

## Student Challenges

1.  **Solvent Flow Rate:** The ratio of solvent to feed is a key design parameter. **Increase the solvent flow rate `S` to `250.0` kg/hr.** Do you need more or fewer stages? What is the practical downside of using more solvent in a real plant (hint: think about having to process the extract stream later)?
2.  **A Better Solvent:** Your R&D department finds a new solvent with a higher affinity for the solute, giving it a **Partition Coefficient `KD` of `8.0`.** How does this impact the number of stages required to achieve the same separation?
3.  **Minimum Solvent Rate:** There is a minimum solvent rate below which the desired separation is impossible, no matter how many stages you have. This occurs when the operating line intersects the equilibrium line. Can you find this minimum solvent rate for the base case?